In [ ]:
from glob import glob
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from copy import copy
from matplotlib import pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import helper
traj = helper.build_muon_test_trajectory()

In [ ]:
from neutrino_level.steps.general_modules.read_data import load_dfs
mc_df, data_df = load_dfs(
    traj, ignore_tables=helper.to_ignore)

In [ ]:
from cr_fluxes import GaisserH4aAugerZombie
import simweights

In [ ]:
proton_flux = simweights.FixedFractionFlux(fractions={2212: 1,
                                                      1000260560: 0},
                                           basis=simweights.GaisserH4a())
iron_flux = simweights.FixedFractionFlux(fractions={2212: 0,
                                                    1000260560: 1},
                                         basis=simweights.GaisserH4a())
auger_flux = GaisserH4aAugerZombie()

weighters = []
index = None

for i, file_i in enumerate(traj.parameters.components.corsika.file_list):
    with pd.HDFStore(file_i, 'r') as hdf_store:
        weighters.append(simweights.CorsikaWeighter(
            hdf_store, traj.parameters.components.corsika.n_files_loaded[i]))
        index_i = hdf_store['MCPrimary'].set_index(
            ['Run', 'Event', 'SubEvent']).index
        if index is None:
            index = index_i
        else:
            index = index.append(index_i)
weighter = np.sum(weighters)
sort_idx = index.argsort()
proton_weights = weighter.get_weights(proton_flux)[sort_idx]
iron_weights = weighter.get_weights(iron_flux)[sort_idx]
auger_weights = weighter.get_weights(auger_flux)[sort_idx]

In [ ]:
mc_df.loc[mc_df['Label'] == 6, 'Weights.GaisserH4a_proton'] = proton_weights
mc_df.loc[mc_df['Label'] == 6, 'Weights.GaisserH4a_iron'] = iron_weights
mc_df.loc[mc_df['Label'] == 6, 'Weights.GaisserH4a_auger'] = auger_weights

In [ ]:
mask = mc_df['Label'] == 6
mc_df.loc[mask, 'Weight'] = mc_df.loc[mask, 'Weights.GaisserH4a']
mask = mc_df['Label'] != 6
mc_df.loc[mask, 'Weight'] = mc_df.loc[mask, 'Weights.northern_tracks'] #* 2

In [ ]:
import sys
sys.path.append('/data/i3home/baclark/IceCube/ehe/career_plots/tools/energy_loss_pdfs')
from stochasticity_observable_ import get_1d_rlogl

In [ ]:
get_1d_rlogl(
    mc_df,
    Likelihood_1D(spline),
    table_name='EHEMuMillipede_SplineMPEseed_vecd',
    key_name='stochasticity',
    min_bins=8,
    n_bins_to_combine=4)

In [ ]:
from neutrino_level.steps.general_modules.label_maker import type_to_label

def get_idx_from_comp(df, comp):
    idx = np.array(df['Label'] == type_to_label[comp], dtype=bool)
    return idx

def calc_weights(
    mc_df, obs_name, component_name,
    weight_col='Weight',
):
    comp_idx = get_idx_from_comp(mc_df, component_name)
    values = mc_df.loc[comp_idx, obs_name]
    weights = mc_df.loc[comp_idx, weight_col]
    
    return values, weights

In [ ]:
cor_values, cor_weights = calc_weights(
    mc_df, 'stochasticity', component_name='corsika'
)
numu_values, numu_weights = calc_weights(
    mc_df, 'stochasticity', component_name='numu_cc'
)

np.savez('ehenextgen_stoch_weights.npz',
         cor_values=cor_values,
         cor_weights=cor_weights,
         numu_values=numu_values,
         numu_weights=numu_weights
         )

In [ ]:
fig, ax = plt.subplots()
ax.hist(
    cor_values, weights=cor_weights,
    bins=np.linspace(2, 14, 101), 
    histtype='step',linewidth=2,
    density=True
)
ax.hist(
    numu_values, weights=numu_weights,
    bins=np.linspace(2, 14, 101), 
    histtype='step',linewidth=2,
    density=True
)
ax.set_yscale('log')
ax.set_ylim([1e-3, 1e0])